In [1]:
import logging
import os

import nivapy3 as nivapy
import whitebox

# Terrain conditioning using Whitebox Tools

An alternative to the PySheds workflow. This notebook processes data for a single catchment. Results seem promising, but at present PySheds is giving better results.

In [2]:
# Choose rasters to process (list of resolutions)
# res_list = [10, 20, 40]
res_list = [40]

# Working directory. Must be on 'shared' to avoid filling the 'user' drive with temporary files
work_dir = r"/home/jovyan/shared/01_datasets/spatial"

log_file = "terrain_conditioning_wbt.log"

In [3]:
# Setup logging
logging.basicConfig(
    filename=log_file,
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %H:%M",
    encoding="utf-8",
    filemode="w",
    level=logging.INFO,
)
print(f"Logging progress to '{log_file}'.")
logging.info("Started")

Logging progress to 'terrain_conditioning_wbt.log'.


In [4]:
def log_callback(msg):
    print(msg)
    logging.info(msg)

In [5]:
%%time

wbt = whitebox.WhiteboxTools()
wbt.set_verbose_mode(True)
wbt.set_compress_rasters(True)
wbt.set_working_dir(work_dir)

temp_fold = os.path.join(work_dir, "temp")
if not os.path.exists(temp_fold):
    os.makedirs(temp_fold)

msg = "Exporting streams to .shp..."
print(msg)
logging.info(msg)
# eng = nivapy.da.connect_postgis()
# riv_gdf = nivapy.da.read_postgis("physical", "norway_nve_elvis_river_network_line", eng)
# riv_gdf = riv_gdf.to_crs("epsg:25833")
riv_shp = os.path.join(temp_fold, "elvis_streams.shp")
# riv_gdf.to_file(riv_shp)

# Process rasters
for res in res_list:
    msg = f"Processing {res} m DTM..."
    print(msg)
    logging.info(msg)
    dem_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/dtm/vassom_234_40m_dtm.tif"
    burn_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_burn/vassom_234_40m_burn.tif"
    fill_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_fill/vassom_234_40m_fill.tif"
    fill_path2 = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_fill/vassom_234_40m_fill2.tif"
    fdir_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_fdir/vassom_234_40m_fdir.tif"
    facc_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_facc/vassom_234_40m_facc.tif"

    msg = "    Burning streams..."
    print(msg)
    logging.info(msg)
    wbt.fill_burn(
        dem_path,
        riv_shp,
        burn_path,
        callback=log_callback,
    )

    # msg = "    Breaching depressions and filling pits..."
    # print(msg)
    # logging.info(msg)
    # wbt.breach_depressions_least_cost(
    #     burn_path,
    #     fill_path,
    #     10000,
    #     max_cost=None,
    #     min_dist=True,
    #     flat_increment=None,
    #     fill=True,
    #     callback=log_callback,
    # )

    msg = "    Filling depressions..."
    print(msg)
    logging.info(msg)
    wbt.fill_depressions(
        burn_path,
        fill_path,
        fix_flats=True,
        flat_increment=None,
        max_depth=None,
        callback=log_callback,
    )

    msg = "    Calculating flow directions..."
    print(msg)
    logging.info(msg)
    wbt.d8_pointer(
        fill_path,
        fdir_path,
        esri_pntr=False,
        callback=log_callback,
    )

    msg = "    Calculating flow accumulation..."
    print(msg)
    logging.info(msg)
    wbt.d8_flow_accumulation(
        fdir_path,
        facc_path,
        out_type="cells",
        log=False,
        clip=False,
        pntr=True,
        esri_pntr=False,
        callback=log_callback,
    )
# shutil.rmtree(temp_fold)

print("Done.")

['./whitebox_tools', '-v']
Exporting streams to .shp...
Processing 40 m DTM...
    Burning streams...
./whitebox_tools --run="FillBurn" --wd="/home/jovyan/shared/01_datasets/spatial" --dem='/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/dtm/vassom_234_40m_dtm.tif' --streams='/home/jovyan/shared/01_datasets/spatial/temp/elvis_streams.shp' --output='/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_40m/by_vassom/wbt_burn/vassom_234_40m_burn.tif' --compress_rasters

****************************
* Welcome to FillBurn      *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading streams data...
Reading DEM data...
Rasterizing Streams: 0%
Rasterizing Streams: 1%
Rasterizing Streams: 2%
Rasterizing Streams: 3%
Rasterizing Streams: 4%
Rasterizing Streams: 5%
Rasterizing Streams: 6%
Rasterizing Streams: 7%
Rasterizing Streams: 8%
Rasterizing Streams: 9%
Rasterizing Streams: 10%
Rasterizing Streams: 11%
Rasterizing St